# Snapshot Analysis Demo (Random Graph)

This notebook illustrates the streamlined `propflow.snapshots` API. We build a random factor graph, run a BP engine for a few iterations, and inspect the recorded snapshots with the analyzer, report helper, and visualizer.

In [ ]:
from pathlib import Path

import numpy as np

from propflow import BPEngine, FGBuilder
from propflow.configs import CTFactory
from propflow.snapshots import SnapshotAnalyzer, AnalysisReport, SnapshotVisualizer


In [ ]:
np.random.seed(7)

graph = FGBuilder.build_random_graph(
    num_vars=6,
    domain_size=3,
    ct_factory=CTFactory.random_int.fn,
    ct_params={'low': 0, 'high': 8},
    density=0.6,
)

engine = BPEngine(factor_graph=graph)
engine.run(max_iter=5)
snapshots = engine.snapshots
len(snapshots)


In [ ]:
first = snapshots[0]
{
    'step': first.step,
    'global_cost': first.global_cost,
    'assignments': first.assignments,
    'q_messages': len(first.Q),
    'r_messages': len(first.R),
}


In [ ]:
analyzer = SnapshotAnalyzer(snapshots)
belief_series = analyzer.beliefs_per_variable()
belief_series


In [ ]:
delta_q, delta_r = analyzer.difference_coordinates(step_idx=0)
{
    'delta_q_keys': list(delta_q)[:3],
    'delta_r_keys': list(delta_r)[:3],
}


In [ ]:
J0 = analyzer.jacobian(step_idx=0)
J0_dense = J0.toarray() if hasattr(J0, 'toarray') else np.asarray(J0)
J0_dense.shape


In [ ]:
cycles = analyzer.cycle_metrics(step_idx=0)
block_norms = analyzer.block_norms(step_idx=0)
nilpotent = analyzer.nilpotent_index(step_idx=0)
{
    'cycle_metrics': cycles,
    'block_norms': block_norms,
    'nilpotent_index': nilpotent,
}


In [ ]:
viz = SnapshotVisualizer(snapshots)
viz.variables(), viz.argmin_series()


In [ ]:
report = AnalysisReport(analyzer)
summary = report.to_json(step_idx=0)
summary


In [ ]:
output_dir = Path('results/snapshot_demo')
report.to_csv(output_dir, step_idx=0)
sorted(p.name for p in output_dir.iterdir())
